# Conformance Checking in Declare4Py

This tutorial explains how to perform the checking of a DECLARE model and how to browse the results.

After importing the Declare4Py package and specified the paths of the log and of the DECLARE model, a `Declare4Py` object has to be instantiated.

In [1]:
import sys
sys.path.append("..")
import os
from src.api.declare4py import Declare4Py


log_path = os.path.join("..", "test", "Sepsis Cases.xes.gz")
model_path = os.path.join("..", "test", "declare_models", "data_model.decl")

d4py = Declare4Py()

The DECLARE constraints supported by Declare4Py can be retrieved with the `get_supported_templates()` function:

In [2]:
print(d4py.get_supported_templates())

('Existence', 'Absence', 'Exactly', 'Init', 'Choice', 'Exclusive Choice', 'Responded Existence', 'Response', 'Alternate Response', 'Chain Response', 'Precedence', 'Alternate Precedence', 'Chain Precedence', 'Not Responded Existence', 'Not Response', 'Not Chain Response', 'Not Precedence', 'Not Chain Precedence')


Notice that for the templates `Existence`, `Absence` and `Exactly` an additional parameter is necessary for the cardinality. This has to be encoded in the DECLARE `.decl` model with a numeric suffix, for example `Exactly2` or `Existence23`.

The next step is the parsing of the log and of the DECLARE model.

In [3]:
d4py.parse_xes_log(log_path)
d4py.parse_decl_model(model_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The DECLARE model can be inspected by getting all the activity names or the constraints. This information is returned as a list of strings.

In [4]:
model_activities = d4py.get_model_activities()
model_constraints = d4py.get_model_constraints()
for idx, act in enumerate(model_activities):
    print(idx, act)
print("-----------------")
for idx, constr in enumerate(model_constraints):
    print(idx, constr)

0 ER Triage
1 ER Registration
2 ER Sepsis Triage
3 Leucocytes
4 CRP
5 LacticAcid
6 IV Antibiotics
7 Admission NC
8 IV Liquid
9 Release A
10 Return ER
11 Admission IC
-----------------
0 Chain Response[Admission NC, Release B] |A.org:group is K |T.org:group is E |
1 Chain Response[Admission NC, Release A] |A.org:group is I |T.org:group is E |133020,957701,s
2 Chain Precedence[IV Liquid, Admission NC] |A.org:group is I |T.org:group is A |92,14473,s
3 Chain Response[ER Registration, ER Triage] |(A.DiagnosticArtAstrup is false) AND (A.SIRSCritHeartRate is true) AND (A.org:group is A) AND (A.DiagnosticBlood is true) AND (A.DisfuncOrg is false) AND (A.DiagnosticECG is true) AND (A.Age >= 45) AND (A.InfectionSuspected is true) AND (A.DiagnosticLacticAcid is true) AND (A.DiagnosticSputum is true) AND (A.Hypoxie is false) AND (A.DiagnosticUrinaryCulture is true) AND (A.DiagnosticLiquor is false) AND (A.SIRSCritTemperature is true) AND (A.Infusion is true) AND (A.Hypotensie is false) AND (A.Diag

Once the log and the DECLARE model are loaded, the `conformance_checking()` function will perform the model checking. The input boolean parameter `consider_vacuity=true` means that vacuously satisfied traces are considered as satisfied, violated otherwise. This function returns a Python dictionary containing the results indexed by trace in the log. Each key is a tuple containing the trace id and its name (that can be retrieved with `get_trace_keys()`). The value is a Python dictionary with keys the names of the constraints (that can be retrieved with `model_constraints()`) and values a `CheckerResult` object containing the number of pendings, activations, violations, fulfilments and the truth value of the trace for that constraint.

```
model_check_res = {trace_1: {constr_1: CheckerResult object, constr_2: CheckerResult object, ...},
                   trace_2: {constr_1: CheckerResult object, ... },
                    ...
                  }
```
The `CheckerResult` objects can be accessed by the attributes `num_pendings`, `num_activations`, `num_fulfillments`, `num_violations` and `state`.

In [5]:
model_check_res = d4py.conformance_checking(consider_vacuity=False)

Computing conformance checking ...


Let's inspect the results for the trace `(1, 'B')` and the constraint
`Chain Response[LacticAcid, Leucocytes] |A.LacticAcid <= 0.8 |T.Leucocytes >= 13.8 |0,2778,m`

In [6]:
constr_id = 11
trace_id = (1, 'B')
print(f"Constraint to check: {model_constraints[constr_id]}")
print(f"Number of pendings: {model_check_res[trace_id][model_constraints[constr_id]].num_pendings}")
print(f"Number of activations: {model_check_res[trace_id][model_constraints[constr_id]].num_activations}")
print(f"Number of fulfilments: {model_check_res[trace_id][model_constraints[constr_id]].num_fulfillments}")
print(f"Number of violation: {model_check_res[trace_id][model_constraints[constr_id]].num_violations}")
print(f"Truth value: {model_check_res[trace_id][model_constraints[constr_id]].state}")

Constraint to check: Chain Response[LacticAcid, Leucocytes] |A.LacticAcid <= 0.8 |T.Leucocytes >= 13.8 |0,2778,m
Number of pendings: 0
Number of activations: 1
Number of fulfilments: 1
Number of violation: 0
Truth value: Satisfied


The checker results can be printed with the `print_conformance_results` function.